### Imports and configs

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator
import torch
import numpy as np
from torchtext import datasets


### plot histogram of data distribution

In [ ]:
preprocd_data = torch.load("artifacts/saved_data/preprocd_data.pt").numpy()
sent_pairs, languages, max_len = preprocd_data.shape
aggregated_data = preprocd_data.reshape([sent_pairs * languages, max_len])

In [ ]:
def plot_token_hist(data):

    # Count occurrences of values equal to 2 and not equal to 2
    count_equal_to_2 = np.sum(data == 2)
    count_not_equal_to_2 = np.sum(data != 2)

    # Plot histogram with two bars
    plt.bar(['Token=2', 'Token!=2'], [count_equal_to_2, count_not_equal_to_2], color=['blue', 'orange'])
    plt.xlabel('Entry')
    plt.ylabel('Count')
    plt.title('Histogram of Token=2 and Token!=2')
    plt.gca().get_yaxis().get_major_formatter().set_scientific(False)
    plt.show()

In [ ]:
plot_token_hist(aggregated_data)

In [ ]:
nonpad_token_count = np.sum(preprocd_data != 2)
pad_token_count = np.sum(preprocd_data == 2)
total_token_count = pad_token_count + nonpad_token_count
percent_nonpad = nonpad_token_count / total_token_count * 100
print(f"nonpad_token_count: {nonpad_token_count}\n"
      f"pad_token_count: {pad_token_count}\n"
      f"total_token_count: {total_token_count}\n"
      f"percent_nonpad: {percent_nonpad:2f} %\n")

In [ ]:
nonpad_sums = np.sum(aggregated_data != 2, axis=1)

In [ ]:
longest_sentence = nonpad_sums.max()
shortest_sentence = nonpad_sums.min()
avg_sentence = np.mean(nonpad_sums)
print(f"longest_sentence: {longest_sentence} tokens\n"
      f"shortest_sentence: {shortest_sentence} tokens\n"
      f"avg_sentence: {avg_sentence:2f} tokens\n"
      )

### histogram of sentence lengths

In [ ]:
def plot_sentence_lens(data, title):
    plt.imshow(data != 2, aspect='auto')
    plt.xlabel("Sentence lengths")
    plt.ylabel("Sentence number")
    plt.title(title)
    plt.grid(True)
    plt.show()

In [ ]:
plot_sentence_lens(preprocd_data[:10000, 0, :], "German sentences")
plot_sentence_lens(preprocd_data[:10000, 1, :], "English sentences")

##### histogram of sentence lengths

In [ ]:
nonpad_token_counts = np.count_nonzero(preprocd_data != 2, axis=2)

In [ ]:
ger_nonpad_token_counts = nonpad_token_counts[:,0]
eng_nonpad_token_counts = nonpad_token_counts[:,1]

In [ ]:
ger_nonpad_token_counts

In [ ]:
fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)
# We can set the number of bins with the *bins* keyword argument.
counts, bins, _ = axs[0].hist(ger_nonpad_token_counts, bins=range(50))
print(f'counts: {counts}')
axs[1].hist(eng_nonpad_token_counts, bins=range(50))
axs[0].set_title('German non-pad token counts')
axs[1].set_title('English non-pad token counts')
axs[0].set_xlabel('Non-pad token counts')
axs[0].set_ylabel(f'Number of sentences (total {len(ger_nonpad_token_counts)})')
axs[1].set_xlabel('Non-pad token counts')
axs[1].set_ylabel(f'Number of sentences (total {len(eng_nonpad_token_counts)})')
plt.plot()

In [ ]:
training_bleu_N1 = []
validation_bleu_N1 = []
with open('training_history_N1.txt') as f:
    lines = f.readlines()
    for line in lines:
        if "Epoch:" in line:
            parts = line.split("|")
            tbleu = float(parts[1].split("BLEU: ")[-1].strip())
            vbleu = float(parts[2].split("BLEU: ")[-1].strip())
            training_bleu_N1.append(tbleu)
            validation_bleu_N1.append(vbleu)


In [ ]:
training_bleu_N1

In [ ]:
plt.figure()
plt.plot(training_bleu_N1, label='training_bleu_N1')
plt.plot(validation_bleu_N1, label='validation_bleu_N1')
plt.legend()

In [ ]:
training_bleu_N6 = []
validation_bleu_N6 = []
with open('training_history_N6.txt') as f:
    lines = f.readlines()
    for line in lines:
        if "Epoch:" in line:
            parts = line.split("|")
            tbleu = float(parts[1].split("BLEU: ")[-1].strip())
            vbleu = float(parts[2].split("BLEU: ")[-1].strip())
            training_bleu_N6.append(tbleu)
            validation_bleu_N6.append(vbleu)

In [ ]:
plt.figure()
plt.plot(training_bleu_N6, label='training_bleu_N6')
plt.plot(validation_bleu_N6, label='validation_bleu_N6')
plt.legend()

In [ ]:
x = datasets.IWSLT2016()

In [ ]:
x

In [ ]:
from datasets import load_dataset

In [ ]:
d = load_dataset('wmt14', 'de-en')

In [ ]:
type(d['test']['translation'])

In [ ]:
for x in d['test']['translation']:
    print(x)
    break

In [ ]:
t,v,te = d['train'], d['validation'], d['test']

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
dl = DataLoader(t)

In [ ]:
for x in dl:
    print(x)
    break

In [ ]:
t

In [ ]:
rd = [tuple(sentence_pair.values()) for sentence_pair in t['translation'] + v['translation'] + te['translation']]

In [ ]:
rd[:2]

In [ ]:
for elem in rd:
    print(elem)
    break

In [ ]:
from torchtext.data.functional import to_map_style_dataset

In [ ]:
for sentence_pair in rd:
    print(sentence_pair)
    src_sentence, tgt_sentence = sentence_pair.values()
    print(src_sentence)
    print(tgt_sentence)
    break

In [ ]:
data_map = to_map_style_dataset(rd)

In [ ]:
for elem in data_map:
    print(elem)
    break